In [11]:
import pandas as pd
import numpy as np

In [12]:
data = pd.read_csv("2017_18_NBA_Stats.csv")
data["Fantasy_points"] = 1.2*data["TRB"] + 1.5*data["AST"] + 3*data["STL"] + 3*data["BLK"] - data["TOV"] + data["PTS"]

In [95]:
aggregate = data.groupby(by=["ID", "Player"], as_index=False).Fantasy_points.agg(["mean", "median"]).reset_index()
aggregate.columns = aggregate.columns.ravel()
aggregate.head()

,ID,Player,mean,median
0,9446,Jeff Teague,31.111429,30.6
1,9447,Joe Johnson,13.234545,12.1
2,9448,Jamal Crawford,15.878750,15.7
3,9449,Josh Smith,2.266667,0.0
4,9451,Marvin Williams,19.957143,19.4


In [197]:
fanduel = pd.read_csv("FanDuel-NBA-2018-10-01-28679-players-list.csv")
fanduel["Id"] = fanduel["Id"].str.split('-').str[1]
fanduel["Id"] = fanduel["Id"].astype(int)
fanduel = pd.merge(fanduel, aggregate, how='left', left_on='Id', right_on='ID')
fanduel.dropna(subset=['FPPG', 'mean'], inplace=True)
fanduel["FPPG"] = fanduel["mean"].astype(str)
fanduel.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,ID,Player,mean,median
0,15755,PF,Anthony,Anthony Davis,Davis,55.04533333333333,76,12000,NO@ATL,NO,ATL,NaN,NaN,NaN,NaN,NaN,15755.0,Anthony Davis,55.045333,56.0
1,49111,C,Joel,Joel Embiid,Embiid,44.33809523809524,65,10400,ORL@PHI,PHI,ORL,NaN,NaN,NaN,NaN,NaN,49111.0,Joel Embiid,44.338095,43.3
2,66115,PG,Ben,Ben Simmons,Simmons,42.1358024691358,82,10200,ORL@PHI,PHI,ORL,NaN,NaN,NaN,NaN,NaN,66115.0,Ben Simmons,42.135802,41.6
3,12332,PG,John,John Wall,Wall,41.73414634146341,41,10000,NY@WAS,WAS,NY,NaN,NaN,NaN,NaN,NaN,12332.0,John Wall,41.734146,44.0
4,63122,PF,Kristaps,Kristaps Porzingis,Porzingis,39.84791666666668,48,8400,NY@WAS,NY,WAS,O,Knee,NaN,NaN,NaN,63122.0,Kristaps Porzingis,39.847917,40.4


In [193]:
fanduel["FPPG"].isna().count()

110

In [198]:
fanduel.to_csv("lineup.csv")

In [199]:
from pydfs_lineup_optimizer import get_optimizer, Site, Sport
optimizer = get_optimizer(Site.FANDUEL, Sport.BASKETBALL)
optimizer.load_players_from_CSV("lineup.csv")

In [200]:
k = 5
exposure = k/k
random = False

for lineup in optimizer.optimize(n=k, max_exposure = exposure, randomness=random):
    print(lineup)
    print(lineup.players)  # list of players
    print(lineup.fantasy_points_projection)
    print(lineup.salary_costs) 

 1. PG    Trey Burke                    PG    NY             23.394  4100.0$   
 2. PG    Bradley Beal                  PG    WAS            37.023  8000.0$   
 3. SG    Austin Rivers                 SG    WAS            26.508  4300.0$   
 4. SG    E'Twaun Moore                 SG    NO             21.416  3600.0$   
 5. SF    Otto Porter Jr.               SF    WAS            30.596  6700.0$   
 6. SF    Tim Hardaway Jr.              SF    NY             28.496  6300.0$   
 7. PF    Anthony Davis                 PF    NO             55.045  12000.0$  
 8. PF    Aaron Gordon                  PF    ORL            34.097  7300.0$   
 9. C     Nikola Vucevic                C     ORL            36.865  7700.0$   

Fantasy Points 293.441
Salary 60000.0
[Trey Burke PG (NY), Bradley Beal PG (WAS), Austin Rivers SG (WAS), E'Twaun Moore SG (NO), Otto Porter Jr. SF (WAS), Tim Hardaway Jr. SF (NY), Anthony Davis PF (NO), Aaron Gordon PF (ORL), Nikola Vucevic C (ORL)]
293.441
60000.0
 1. PG    Tr